## Amazon SageMaker DICOM Training Overview

In this example we will demonstrate how to integrate the [MONAI](http://monai.io) framework into Amazon SageMaker, use SageMaker Ground Truth labelled data, and give example code of MONAI pre-processing transforms and neural network (DenseNet) that you can use to train a medical image classification model using DICOM images directly.  Please also visit [Build a medical image analysis pipeline on Amazon SageMaker using the MONAI framework](https://aws.amazon.com/blogs/industries/build-a-medical-image-analysis-pipeline-on-amazon-sagemaker-using-the-monai-framework/) for additional details on how to deploy the MONAI model, pipe input data from S3, and perform batch inferences using SageMaker batch transform.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

In [ ]:
# Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0

!pip install -r ./source/requirements.txt
!mkdir -p data

In [55]:
import os
from pathlib import Path
from dotenv import load_dotenv
env_path = Path('.') / 'set.env'
load_dotenv(dotenv_path=env_path)

bucket=os.environ.get('BUCKET')
bucket_path=os.environ.get('BUCKET_PATH')
user=os.environ.get('DICOM_USER')
password = os.environ.get('DICOM_PASSWORD')

print('Bucket: '+bucket)
print('Bucket Prefix: '+bucket_path)
print('User: '+user)
print('Password: '+password)

Bucket: sm-gt-orthanc-dicom
Bucket Prefix: output/AnnotateDICOMfromOrthanc-checkbox-2/annotations/consolidated-annotation/consolidation-request/iteration-1/2021-01-15_05:22:56.json
User: orthanc
Password: orthanc


## Amazon SageMaker GroundTruth Labeling Metadata

Download and parse the GroundTruth JSON annotation file for labelled data, classes and DICOM image URLs. 

In [56]:
import json
import boto3

image_url_list = []
image_label_list = []

# Get SageMaker GroundTruth labeling data from annotation file
datadir = './data'
metadata = datadir+'/meta-data.json'
s3 = boto3.client('s3')
s3.download_file(bucket, bucket_path, metadata)

# Load Labels
with open(metadata) as f:
    manifest = json.load(f)
    class_names = list(json.loads(manifest[0]['annotations'][0]['annotationData']['content'])['disease'].keys())
    
for i, j in enumerate(manifest):
    label_dict = json.loads(json.loads(manifest[i]['annotations'][0]['annotationData']['content'])['labels'])
    image_url_list.extend([label_dict['imageurl']])
    image_label_list.extend([class_names.index(label_dict['label'][0])])

## DICOM Upload Files to S3

Download the DICOM files from Orthanc DICOM Server into S3 for SageMaker training.

In [ ]:
import urllib3
import requests
import os
from io import BytesIO
import contextlib;
urllib3.disable_warnings()

image_file_list = []

#Load DICOM images to S3
for i, j in enumerate(image_url_list):
    file_name = image_url_list[i].split("/file")[0].split("instances/")[1] + '.dcm'
    response = requests.get(image_url_list[i], auth=(user, password), stream=True, verify=False)
    fp = BytesIO(response.content)
    s3.upload_fileobj(fp, bucket, file_name)   
    image_file_list.append(file_name)

## DICOM Display Sample Set

Download a sample of DICOM images from the S3 bucket and display with label from annotation file.

In [ ]:
import monai
from monai.transforms import Compose, LoadImage, Resize, ScaleIntensity, ToTensor, SqueezeDim
import matplotlib.pyplot as plt

#Display sample of DICOM Images
trans = Compose([LoadImage(image_only=True), Resize(spatial_size=(108,96)),SqueezeDim()])
plt.subplots(1, 3, figsize=(8, 8))
for i in range(0,3):
    s3.download_file(bucket, image_file_list[i], datadir+'/'+image_file_list[i])
    img = trans(datadir+'/'+image_file_list[i])
    plt.subplot(1, 3, i + 1)
    plt.xlabel(class_names[image_label_list[i]])
    plt.imshow(img, cmap='gray')
plt.tight_layout()
plt.show()

## Data

### Create Sagemaker session and S3 location for DICOM dataset

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

inputs = sagemaker_session.upload_data(path=datadir, bucket=bucket)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## Train Model
### Training

The ```monai_dicom.py``` script provides all the code we need for training and hosting a SageMaker model (model_fn function to load a model). The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* SM_MODEL_DIR: A string representing the path to the directory to write model artifacts to. These artifacts are uploaded to S3 for model hosting.
* SM_NUM_GPUS: The number of gpus available in the current container.
* SM_CURRENT_HOST: The name of the current container on the container network.
* SM_HOSTS: JSON encoded list containing all the hosts .
Supposing one input channel, 'training', was used in the call to the PyTorch estimator's fit() method, the following will be set, following the format SM_CHANNEL_[channel_name]:

* SM_CHANNEL_TRAINING: A string representing the path to the directory containing data in the 'training' channel.
For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to model_dir so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an argparse.ArgumentParser instance.

In [ ]:
!pygmentize source/monai_dicom.py

## Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure.  We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters.  In this case we are going to run our training job on ```ml.m5.xlarge``` instance.  But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)).  The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the ```monai_dicom.py``` script above.

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='monai_dicom.py',
                    source_dir='source',
                    role=role,
                    framework_version='1.5.0',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.m5.xlarge',
                    hyperparameters={
                        'backend': 'gloo',
                        'epochs': 5
                    })

After we've constructed our PyTorch object, we can fit it using the DICOM dataset we uploaded to S3.

In [ ]:
estimator.fit({'train': inputs})